In [220]:
import numpy as np
import math
from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, limited_depth_first, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import random

In [221]:
crater_map = np.load('crater_map.npy')

# Obtener numero de filas (nr) y numero de columnas (nc) de la configuracion del mapa
nr, nc = crater_map.shape

In [222]:
# Escala (metros por cada pixel del mapa)
escala = 10.045

# Funcion para calcular la fila en base a su coordenada 'y' en el mapa
def get_row(nr,y):
  """
    Calcula la fila correspondiente a una coordenada 'y' en el mapa.

      nr : Numero total de filas en el mapa.
      y : Coordenada y en el sistema de coordenadas de la imagen.

    Retorna:
        int: Fila correspondiente a la coordenada 'y' en el mapa.
    """
  return int(nr-np.round(y/escala,decimals=0))

# Funcion para calcular la columna en base a su coordenada 'x' en el mapa
def get_col(x):
  """
    Calcula la columna correspondiente a una coordenada 'x' en el mapa.

    Parametros:
        x (float): Coordenada x en el sistema de coordenadas de la imagen.

    Retorna:
        int: Columna correspondiente a la coordenada x en el mapa.
   """
  return int(np.round(x/escala,decimals=0))

In [223]:
def get_y(nr, row):
    """
    Calcula la coordenada 'y' correspondiente a una fila en el mapa.

    Parámetros:
        nr (int): Número total de filas en el mapa.
        row (int): Fila en el mapa.

    Retorna:
        float: Coordenada y correspondiente.
    """
    return (nr - row) * escala

def get_x(col):
    """
    Calcula la coordenada 'x' correspondiente a una columna en el mapa.

    Parámetros:
        col (int): Columna en el mapa.

    Retorna:
        float: Coordenada x correspondiente.
    """
    return col * escala


In [224]:
class Rover():
    '''Clase utilizada para describir el problema del Rover descendiendo el crater. Los estados son representados
        por medio de la posicion del Rover en la imagen.'''

    def __init__(self, x, y, crater_map):
        '''
        Constructor de la clase Rover.

        Parametros:
        - x: Coordenada 'x' inicial del robot en el mapa de Marte.
        - y: Coordenada 'y 'inicial del robot en el mapa de Marte.
        - crater_map: Mapa del crater de Marte representado como un arreglo.

        Inicializa las coordenadas 'x' e 'y' del robot en el mapa de Marte y guarda una referencia al mapa del cráter.
        '''
        self.x = x
        self.y = y
        self.crater_map = crater_map

    def cost(self, x, y):
        '''
        Parametros:
        - x: Coordenada 'x' en la posicion deseada del crater.
        - y: Coordenada 'y' en la posicion deseada del crater.

        Retorna:
        La altura en la posición dada del crater.

        Este método calcula la altura en una posición específica del cráter de Marte.
        '''
        return self.crater_map[y, x]

    def best_neighbor(self, current_x, current_y):
        '''
        Este método busca el mejor vecino para moverse desde la posición actual del robot en el cráter.

        Parámetros:
        - current_x: Coordenada x actual del robot en el cráter.
        - current_y: Coordenada y actual del robot en el cráter.

        Retorna:
        Las coordenadas del mejor vecino para moverse desde la posicion actual del robot en el crater.
        '''
        min_cost = float('inf')
        best_move = None

        for dx in range(-1, 2):
            for dy in range(-1, 2):
                new_x = current_x + dx
                new_y = current_y + dy

                if 0 <= new_x < self.crater_map.shape[1] and 0 <= new_y < self.crater_map.shape[0]:
                    # Calcular el costo en la nueva posición
                    new_cost = self.cost(new_x, new_y)

                    # Verificar si se mejora y si no existe una diferencia mayor a 2 metros
                    if new_cost < min_cost and abs(new_cost - self.cost(current_x, current_y)) <= 2.0:
                        min_cost = new_cost
                        best_move = (new_x, new_y)

        return best_move
    def neighbor(self):
        """
        Selecciona un vecino de alrededor con la condicion de que la diferencia de alturas
        entre la posición actual y la del vecino no sea mayor a 2.0 metros.
        Retorna:
        Las coordenadas de un vecino aleatorio que cumpla con la condicion dada
        """
        neighbors = []
        directions = [(dx,dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if (dx,dy) != (0,0)]
        current_cost = self.cost(self.x, self.y)

        # Generar vecindad
        for i, j in directions:  
            dx, dy = i, j
            new_x, new_y = self.x + dx, self.y + dy

            # Verificar si las nuevas coordenadas están dentro de los límites del mapa
            if 0 <= new_x < self.crater_map.shape[1] and 0 <= new_y < self.crater_map.shape[0]:
                new_cost = self.cost(new_x, new_y)

                # Verificar si la diferencia de alturas no es mayor a 2.0 metros
                if abs(new_cost - current_cost) <= 2.0:
                    neighbors.append((new_x, new_y))

        # Seleccionar un vecino aleatorio de los que cumplen la condición
        if neighbors:
            return random.choice(neighbors)
        else:
            return None



In [225]:
# Calcula el número de renglón correspondiente a la coordenada y inicial
row_inicial = get_row(nr, y_inicial)
# Calcula el número de columna correspondiente a la coordenada x inicial
col_inicial = get_col(x_inicial)
# Estado inicial del robot representado como una tupla de (columna, renglón)
estado_inicial = (col_inicial, row_inicial)

In [226]:
#Definicion del algoritmo 'Hill Climbing' o Busqueda Voraz
def hill_climbing(estado_inicial, x_inicial, y_inicial, crater_map):
    """
    Parametros:
        estado_inicial : Coordenadas (x, y) de la posición inicial.
        x_inicial : Coordenada x inicial.
        y_inicial : Coordenada y inicial.
        crater_map : Mapa del crater.

    Retorna:
        tuple: Coordenadas (x, y) de la posición final y altura final alcanzada.
    """
    exploration = Rover(estado_inicial[0], estado_inicial[1], crater_map)

    coordenadas_x = [x_inicial]
    coordenadas_y = [y_inicial]

    # Calcular el costo inicial
    cost = exploration.cost(estado_inicial[0], estado_inicial[1])

    print("**********Hill Climbing**********")
    print('-------- Initial state -----------')
    print("Altura inicial: ", round(cost, 2), 'metros')
    print("Posicion inicial: ", estado_inicial, 'px')
    print("Coordenadas iniciales: ", (x_inicial, y_inicial), '(x, y)')

    print("\n--------Iterations--------")

    step = 0  # Contador de pasos
    while True:
        step += 1
        # Obtener el mejor vecino
        neighbor = exploration.best_neighbor(exploration.x, exploration.y)
        if neighbor is None:
            break  # No hay vecinos disponibles, salir del bucle
        new_cost = exploration.cost(neighbor[0], neighbor[1])
        # Probar el vecino
        if new_cost < cost:
            (exploration.x, exploration.y) = neighbor  # Actualizar la posición del explorador
            cost = new_cost
        else:
            break
        print ("\nCoordenadas (x, y): ", (round(get_x(neighbor[0])),round(get_y(nr, neighbor[1]))))
        coordenadas_x.append(get_x(exploration.x))
        coordenadas_y.append(get_y(nr, exploration.y))
        print("Step ", step, " Altura: ", round(cost, 2), "metros")

    altura_final = crater_map[exploration.x,  exploration.y]

    print("\n--------Solution found-----------")
    print("Altura final: ", round(cost, 2), 'metros')
    print("Posicion Final:", (exploration.x, exploration.y), 'px')
    print("Coordenadas finales:",(round(get_x(exploration.x)),round(get_y(nr, exploration.y))), '(x, y)')



In [227]:
def calcular_posicion_matriz(x_inicial, y_inicial, nr):
    """
    Calcula las coordenadas iniciales del rover dentro del mapa.

    Parámetros:
        x_inicial: Coordenada x inicial.
        y_inicial : Coordenada y inicial.
        nr : Número total de filas en el mapa.

    Retorna:
        tuple: Coordenadas iniciales del robot representadas como una tupla de (columna, renglón).
    """
    # Calcula el número de renglón correspondiente a la coordenada y inicial
    row_inicial = get_row(nr, y_inicial)
    # Calcula el número de columna correspondiente a la coordenada x inicial
    col_inicial = get_col(x_inicial)
    # Estado inicial del robot representado como una tupla de (columna, renglón)
    estado_inicial = (col_inicial, row_inicial)

    return estado_inicial

In [228]:
#Posicion de primer caso de prueba con 'Hill Climbing'
x_inicial = 3350
y_inicial = 5800


estado_inicial = calcular_posicion_matriz(x_inicial, y_inicial, nr)
hill_climbing(estado_inicial, x_inicial, y_inicial, crater_map)

**********Hill Climbing**********
-------- Initial state -----------
Altura inicial:  117.03 metros
Posicion inicial:  (333, 500) px
Coordenadas iniciales:  (3350, 5800) (x, y)

--------Iterations--------

Coordenadas (x, y):  (3345, 5786)
Step  1  Altura:  115.07 metros

Coordenadas (x, y):  (3335, 5786)
Step  2  Altura:  114.34 metros

Coordenadas (x, y):  (3325, 5776)
Step  3  Altura:  113.81 metros

Coordenadas (x, y):  (3315, 5776)
Step  4  Altura:  113.15 metros

Coordenadas (x, y):  (3305, 5776)
Step  5  Altura:  112.7 metros

Coordenadas (x, y):  (3295, 5776)
Step  6  Altura:  111.26 metros

Coordenadas (x, y):  (3305, 5766)
Step  7  Altura:  109.95 metros

Coordenadas (x, y):  (3305, 5756)
Step  8  Altura:  109.62 metros

--------Solution found-----------
Altura final:  109.62 metros
Posicion Final: (329, 504) px
Coordenadas finales: (3305, 5756) (x, y)


In [229]:
#Posicion de segundo caso de prueba con 'Hill Climbing'
x_2 = 3890
y_2 = 5510

estado_inicial = calcular_posicion_matriz(x_2, y_2, nr)
hill_climbing(estado_inicial, x_2, y_2, crater_map)

**********Hill Climbing**********
-------- Initial state -----------
Altura inicial:  126.63 metros
Posicion inicial:  (387, 528) px
Coordenadas iniciales:  (3890, 5510) (x, y)

--------Iterations--------

Coordenadas (x, y):  (3887, 5505)
Step  1  Altura:  125.13 metros

Coordenadas (x, y):  (3877, 5515)
Step  2  Altura:  123.33 metros

Coordenadas (x, y):  (3867, 5525)
Step  3  Altura:  121.8 metros

Coordenadas (x, y):  (3867, 5515)
Step  4  Altura:  119.98 metros

Coordenadas (x, y):  (3867, 5505)
Step  5  Altura:  118.11 metros

Coordenadas (x, y):  (3857, 5505)
Step  6  Altura:  116.15 metros

Coordenadas (x, y):  (3847, 5495)
Step  7  Altura:  115.27 metros

Coordenadas (x, y):  (3847, 5485)
Step  8  Altura:  115.13 metros

Coordenadas (x, y):  (3857, 5475)
Step  9  Altura:  114.72 metros

Coordenadas (x, y):  (3867, 5464)
Step  10  Altura:  113.67 metros

--------Solution found-----------
Altura final:  113.67 metros
Posicion Final: (385, 533) px
Coordenadas finales: (3867, 546

In [230]:
#Posicion de tercer caso de prueba con 'Hill Climbing'
x_3 = 6100
y_3 = 9160

estado_inicial = calcular_posicion_matriz(x_3, y_3, nr)
hill_climbing(estado_inicial, x_3, y_3, crater_map)

**********Hill Climbing**********
-------- Initial state -----------
Altura inicial:  141.46 metros
Posicion inicial:  (607, 165) px
Coordenadas iniciales:  (6100, 9160) (x, y)

--------Iterations--------

Coordenadas (x, y):  (6087, 9171)
Step  1  Altura:  141.41 metros

Coordenadas (x, y):  (6087, 9181)
Step  2  Altura:  141.31 metros

Coordenadas (x, y):  (6097, 9191)
Step  3  Altura:  141.27 metros

Coordenadas (x, y):  (6107, 9201)
Step  4  Altura:  141.19 metros

Coordenadas (x, y):  (6107, 9211)
Step  5  Altura:  141.1 metros

Coordenadas (x, y):  (6097, 9221)
Step  6  Altura:  140.87 metros

Coordenadas (x, y):  (6107, 9231)
Step  7  Altura:  140.75 metros

Coordenadas (x, y):  (6097, 9241)
Step  8  Altura:  140.58 metros

Coordenadas (x, y):  (6087, 9251)
Step  9  Altura:  140.4 metros

Coordenadas (x, y):  (6097, 9261)
Step  10  Altura:  140.3 metros

Coordenadas (x, y):  (6097, 9272)
Step  11  Altura:  140.17 metros

Coordenadas (x, y):  (6097, 9282)
Step  12  Altura:  140.0

In [231]:
#Posicion de cuarto caso de prueba con 'Hill Climbing'
x_4 = 2360
y_4 = 5050

estado_inicial = calcular_posicion_matriz(x_4, y_4, nr)
hill_climbing(estado_inicial, x_4, y_4, crater_map)

**********Hill Climbing**********
-------- Initial state -----------
Altura inicial:  74.44 metros
Posicion inicial:  (235, 574) px
Coordenadas iniciales:  (2360, 5050) (x, y)

--------Iterations--------

Coordenadas (x, y):  (2371, 5043)
Step  1  Altura:  73.99 metros

Coordenadas (x, y):  (2381, 5043)
Step  2  Altura:  73.34 metros

Coordenadas (x, y):  (2391, 5033)
Step  3  Altura:  72.51 metros

Coordenadas (x, y):  (2401, 5022)
Step  4  Altura:  71.1 metros

Coordenadas (x, y):  (2411, 5022)
Step  5  Altura:  70.1 metros

Coordenadas (x, y):  (2421, 5022)
Step  6  Altura:  69.61 metros

Coordenadas (x, y):  (2431, 5022)
Step  7  Altura:  69.39 metros

Coordenadas (x, y):  (2441, 5012)
Step  8  Altura:  68.99 metros

Coordenadas (x, y):  (2451, 5002)
Step  9  Altura:  67.09 metros

Coordenadas (x, y):  (2451, 4992)
Step  10  Altura:  65.25 metros

Coordenadas (x, y):  (2461, 5002)
Step  11  Altura:  63.61 metros

--------Solution found-----------
Altura final:  63.61 metros
Posicio

In [232]:
#Posicion de quinto caso de prueba con 'Hill Climbing'
x_5 = 3400
y_5 = 6750

estado_inicial = calcular_posicion_matriz(x_5, y_5, nr)
hill_climbing(estado_inicial, x_5, y_5, crater_map)

**********Hill Climbing**********
-------- Initial state -----------
Altura inicial:  133.41 metros
Posicion inicial:  (338, 405) px
Coordenadas iniciales:  (3400, 6750) (x, y)

--------Iterations--------

Coordenadas (x, y):  (3405, 6760)
Step  1  Altura:  132.02 metros

Coordenadas (x, y):  (3415, 6760)
Step  2  Altura:  130.28 metros

Coordenadas (x, y):  (3425, 6750)
Step  3  Altura:  128.31 metros

Coordenadas (x, y):  (3435, 6750)
Step  4  Altura:  126.89 metros

Coordenadas (x, y):  (3445, 6760)
Step  5  Altura:  125.95 metros

Coordenadas (x, y):  (3455, 6760)
Step  6  Altura:  125.16 metros

Coordenadas (x, y):  (3466, 6770)
Step  7  Altura:  124.61 metros

Coordenadas (x, y):  (3476, 6770)
Step  8  Altura:  124.08 metros

Coordenadas (x, y):  (3486, 6770)
Step  9  Altura:  123.69 metros

Coordenadas (x, y):  (3496, 6780)
Step  10  Altura:  123.32 metros

--------Solution found-----------
Altura final:  123.32 metros
Posicion Final: (348, 402) px
Coordenadas finales: (3496, 67

In [233]:
#Posicion de sexto caso de prueba con 'Hill Climbing'
x_6 = 5100
y_6 = 9830

estado_inicial = calcular_posicion_matriz(x_6, y_6, nr)
hill_climbing(estado_inicial, x_6, y_6, crater_map)

**********Hill Climbing**********
-------- Initial state -----------
Altura inicial:  137.5 metros
Posicion inicial:  (508, 98) px
Coordenadas iniciales:  (5100, 9830) (x, y)

--------Iterations--------

Coordenadas (x, y):  (5113, 9834)
Step  1  Altura:  137.37 metros

Coordenadas (x, y):  (5123, 9834)
Step  2  Altura:  137.36 metros

Coordenadas (x, y):  (5133, 9844)
Step  3  Altura:  137.31 metros

Coordenadas (x, y):  (5133, 9854)
Step  4  Altura:  137.24 metros

Coordenadas (x, y):  (5133, 9864)
Step  5  Altura:  137.17 metros

Coordenadas (x, y):  (5143, 9874)
Step  6  Altura:  137.0 metros

Coordenadas (x, y):  (5153, 9884)
Step  7  Altura:  136.76 metros

--------Solution found-----------
Altura final:  136.76 metros
Posicion Final: (513, 93) px
Coordenadas finales: (5153, 9884) (x, y)


In [234]:
#Definicion del algoritmo 'Recocido Simulado'
def recocido_simulado(estado_inicial, x_inicial, y_inicial, crater_map):
  """
    Parametros:
        estado_inicial : Coordenadas (x, y) de la posicion inicial.
        x_inicial : Coordenada x inicial.
        y_inicial : Coordenada y inicial.
        crater_map : Mapa del crater.

    Retorna:
        tuple: Coordenadas (x, y) de la posición final y altura final alcanzada.
    """
  exploration = Rover(estado_inicial[0], estado_inicial[1], crater_map)

  coordenadas_x = [x_inicial]
  coordenadas_y = [y_inicial]
  # Calcular el costo inicial
  cost = exploration.cost(estado_inicial[0], estado_inicial[1])

  print("**********Recocido Simulado**********")
  print("Altura inicial: ", round(cost, 2), 'metros')
  print("Posicion inicial:", estado_inicial, 'px')
  print("Coordenadas iniciales: ", (x_inicial, y_inicial), '(x, y)')

  step = 0      
  alpha = 0.9998              # Coefficient of the exponential temperature schedule
  t0 = 2                      # Initial temperature
  t = t0

  while t > 0.005 and cost > 0:
    t = t0 * alpha ** step
    step += 1

    # Obtener un vecino aleatorio
    neighbor = exploration.neighbor()

    if neighbor is not None:
        new_cost = exploration.cost(neighbor[0], neighbor[1])

        # Calcular la diferencia de alturas
        delta_cost = new_cost - cost

        if delta_cost < 0 or random.uniform(0, 1) < math.exp(-delta_cost / t):
            exploration.x, exploration.y = neighbor
            cost = new_cost
    coordenadas_x.append(get_x(exploration.x))
    coordenadas_y.append(get_y(nr, exploration.y))

  print("\nTotal de iteraciones: ", step)
  print("\n--------Solution found-----------")
  print("Altura final: ", round(cost, 2), 'metros')
  print("Posicion final: ", (exploration.x, exploration.y), 'px')
  print("Coordenadas finales:",(round(get_x(exploration.x)),round(get_y(nr, exploration.y))), '(x, y)')
  print("\n")


In [235]:
#Posicion de primer caso de prueba con 'Recocido Simulado'
x_inicial = 3350
y_inicial = 5800


estado_inicial = calcular_posicion_matriz(x_inicial, y_inicial, nr)
recocido_simulado(estado_inicial, x_inicial, y_inicial, crater_map)

**********Recocido Simulado**********
Altura inicial:  117.03 metros
Posicion inicial: (333, 500) px
Coordenadas iniciales:  (3350, 5800) (x, y)

Total de iteraciones:  29956

--------Solution found-----------
Altura final:  0.27 metros
Posicion final:  (294, 584) px
Coordenadas finales: (2953, 4952) (x, y)




In [236]:
#Posicion de segundo caso de prueba con 'Recocido Simulado'
x_2 = 3890
y_2 = 5510

estado_inicial = calcular_posicion_matriz(x_2, y_2, nr)
recocido_simulado(estado_inicial, x_2, y_2, crater_map)

**********Recocido Simulado**********
Altura inicial:  126.63 metros
Posicion inicial: (387, 528) px
Coordenadas iniciales:  (3890, 5510) (x, y)

Total de iteraciones:  29956

--------Solution found-----------
Altura final:  85.84 metros
Posicion final:  (366, 570) px
Coordenadas finales: (3676, 5093) (x, y)




In [237]:
#Posicion de tercer caso de prueba con 'Recocido Simulado'
x_3 = 6100
y_3 = 9160

estado_inicial = calcular_posicion_matriz(x_3, y_3, nr)
recocido_simulado(estado_inicial, x_3, y_3, crater_map)

**********Recocido Simulado**********
Altura inicial:  141.46 metros
Posicion inicial: (607, 165) px
Coordenadas iniciales:  (6100, 9160) (x, y)

Total de iteraciones:  29956

--------Solution found-----------
Altura final:  133.79 metros
Posicion final:  (580, 63) px
Coordenadas finales: (5826, 10186) (x, y)




In [238]:
#Posicion de cuarto caso de prueba con 'Recocido Simulado'
x_4 = 2360
y_4 = 5050

estado_inicial = calcular_posicion_matriz(x_4, y_4, nr)
recocido_simulado(estado_inicial, x_4, y_4, crater_map)

**********Recocido Simulado**********
Altura inicial:  74.44 metros
Posicion inicial: (235, 574) px
Coordenadas iniciales:  (2360, 5050) (x, y)

Total de iteraciones:  29956

--------Solution found-----------
Altura final:  55.46 metros
Posicion final:  (247, 579) px
Coordenadas finales: (2481, 5002) (x, y)




In [239]:
#Posicion de quinto caso de prueba con 'Recocido Simulado'
x_5 = 3400
y_5 = 6750

estado_inicial = calcular_posicion_matriz(x_5, y_5, nr)
recocido_simulado(estado_inicial, x_5, y_5, crater_map)

**********Recocido Simulado**********
Altura inicial:  133.41 metros
Posicion inicial: (338, 405) px
Coordenadas iniciales:  (3400, 6750) (x, y)

Total de iteraciones:  29956

--------Solution found-----------
Altura final:  123.29 metros
Posicion final:  (350, 400) px
Coordenadas finales: (3516, 6800) (x, y)




In [242]:
#Posicion de sexto caso de prueba con 'Recocido Simulado'
x_6 = 5100
y_6 = 9830

estado_inicial = calcular_posicion_matriz(x_6, y_6, nr)
recocido_simulado(estado_inicial, x_6, y_6, crater_map)

**********Recocido Simulado**********
Altura inicial:  137.5 metros
Posicion inicial: (508, 98) px
Coordenadas iniciales:  (5100, 9830) (x, y)

Total de iteraciones:  29956

--------Solution found-----------
Altura final:  132.91 metros
Posicion final:  (488, 10) px
Coordenadas finales: (4902, 10718) (x, y)


